In [ ]:
!pip install scikeras

In [55]:
# --- Python Module Imports ---
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.datasets import mnist
from scikeras.wrappers import KerasClassifier

In [56]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [49]:
# One Hot Encoding the training and test data for the target variable
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [57]:
'''
Create Convolutional Neural Network(CNN) that will be trained from the MNIST
digits dataset
'''

def createCNN(numFilters, learnRate):
  cnnModel = Sequential()

  # Convolutional layer with filters
  cnnModel.add(Conv2D(filters=numFilters, kernel_size=(3,3), activation='relu',
                      input_shape=(28, 28, 1)))

  # Maxpooling layer
  cnnModel.add(MaxPooling2D(pool_size=(2,2)))

  # Flattening the output giving it to fully connected layer
  cnnModel.add(Flatten())

  # Hidden layer maps flattened neurons to output
  cnnModel.add(Dense(10, activation='softmax'))
  cnnModel.compile(optimizer=Adam(learning_rate=learnRate),
                   loss='categorical_crossentropy', metrics=['accuracy'])

  return cnnModel

In [ ]:
# Hyperparameters of CNN(number of filters, learning rate)
paramGrid = {
    'filters': [16,32],
    'learning_rate': [0.001, 0.01]
}

# 5-fold Stratified Cross Validation
stratKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# CNN Hyperparameter Variables
mean_f1Scores = {}
test = 0
highestF1Score = 0
best_params = ()

# Test CNN model for each unique set of hyperparameters
for filters in paramGrid['filters']:
  for learnRate in paramGrid['learning_rate']:
    # Retrieve F1 scores for each fold of the current CNN
    f1Scores = []

    # Store parameters of CNN in case the CNN model achieves the highest F1 score
    parameters = (filters, learnRate)

    for trainIdxes, validateIdxes in stratKFold.split(X_train, y_train):
      X_train_fold = X_train[trainIdxes]
      y_train_fold = y_train_encoded[trainIdxes]
      X_validate_fold = X_train[validateIdxes]
      y_validate_fold = y_train_encoded[validateIdxes]

      # Create pipeline for the CNN model with Keras Classifier
      cnnPipeline = Pipeline([
          ('classifier', KerasClassifier(createCNN, numFilters=filters, learnRate=learnRate))
      ])
      # Fit the training data of the fold into the CNN model
      cnnPipeline.fit(X_train_fold, y_train_fold)

      # Give predictions on what the target values should be for the validation features
      y_predict = cnnPipeline.predict(X_validate_fold)

      # Get the corresponding F1 score for the CNN model with the predicted and validated target data
      f1Score = f1_score(np.argmax(y_validate_fold, axis=1), np.argmax(y_predict, axis=1), average='weighted')
      f1Scores.append(f1Score)

    # Get Mean F1 Score of the hyperparameter set
    meanF1Score = np.mean(f1Scores)

    # Update best parameters for CNN model and its corresponding mean F1 score
    if meanF1Score > highestF1Score:
      highestF1Score = meanF1Score
      best_params = parameters

In [ ]:
# Display CNN model with best parameters and its corresponding F1 Score
print("The best parameters are " + str(best_params[0]) +
      " filters at a learning rate of " + str(best_params[1]))
print("The highest Mean F1 score is " + str(highestF1Score))

In [ ]:
### Train Network with All Training Data using the Best Parameters and Validate with the Test Data ###

# Train Network with All the Training Data
bestCNNModel = createCNN(best_params[0], best_params[1])
bestCNNModel.fit(X_train, y_train_encoded)
y_predict_test = bestCNNModel.predict(X_test)

# Get F1 score of CNN model with the test data
f1ScoreTest = f1_score(np.argmax(y_test_encoded, axis=1), np.argmax(y_predict_test, axis=1), average='weighted')
print("The Mean F1 score of the Best CNN Model on this testing dataset is "
  + str(f1ScoreTest))